In [146]:
import pandas as pd
import numpy as np
import datetime
import time
import json
from geojson import Feature, FeatureCollection, Point

# Predictions Data

In [187]:
original_predictions = pd.read_csv('../notebooks/final_result_2022_07_05-2022_07_30.csv' )
original_predictions.head()

,Unnamed: 0,day0,day_confidence0,day1,day_confidence1,day2,day_confidence2,day3,day_confidence3,day4,...,day22,day_confidence22,day23,day_confidence23,day24,day_confidence24,day25,day_confidence25,latitude,longitude
0,0,0,0.0,0,0.0,0,0.000000,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.727215
1,1,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.714526
2,2,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.676457
3,3,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.625699
4,4,0,0.0,0,0.0,1,0.342601,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.613010


In [189]:
original_predictions = original_predictions.iloc[:, -12:]
original_predictions

,day21,day_confidence21,day22,day_confidence22,day23,day_confidence23,day24,day_confidence24,day25,day_confidence25,latitude,longitude
0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.727215
1,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.714526
2,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.676457
3,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.625699
4,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.613010
...,...,...,...,...,...,...,...,...,...,...,...,...
129799,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.555160,-122.245014
129800,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.359220
129801,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.333841
129802,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.321151


In [190]:
column_names = []

for i in range(5):
    column_names.extend([f'day{i}', f'day_confidence{i}'])
column_names.extend(['latitude', 'longitude'])

original_predictions.columns = pd.Series(column_names)
original_predictions

,day0,day_confidence0,day1,day_confidence1,day2,day_confidence2,day3,day_confidence3,day4,day_confidence4,latitude,longitude
0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.727215
1,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.714526
2,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.676457
3,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.625699
4,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,49.006704,-122.613010
...,...,...,...,...,...,...,...,...,...,...,...,...
129799,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.555160,-122.245014
129800,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.359220
129801,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.333841
129802,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.542471,-122.321151


In [191]:
cur_date = datetime.date(2022, 7, 26)
delta = datetime.timedelta(days = 1)


In [193]:
days = []

for i in range(5):
    cur_day = (original_predictions[[f"day{i}", f"day_confidence{i}", "latitude", "longitude"]]).dropna()
    cur_date_col = pd.DataFrame([cur_date]*len(cur_day.index), columns=["date"])
    cur_day = cur_date_col.join(cur_day)
    cur_date += delta
    # curDay = curDay.nlargest(500, f'day{i}_confidence')
    cur_day.columns = pd.Series(['date', 'dayResult', 'dayConfidence', 'latitude', 'longitude'])
    cur_day = cur_day[cur_day['dayResult'] != 0]
    days.append(cur_day)



In [194]:
all_predictions = pd.concat(days)
all_predictions = all_predictions[all_predictions["dayResult"] == 1]
all_predictions.insert(0, "SNo", range(0, len(all_predictions)))
all_predictions["dateUnixTimestamp"] = all_predictions.apply( lambda r: int(time.mktime(r["date"].timetuple())), axis=1 )
all_predictions.drop(columns=['date', 'SNo', 'dayResult'], inplace=True)
all_predictions.reset_index(drop=True, inplace=True)
all_predictions

,dayConfidence,latitude,longitude,dateUnixTimestamp
0,0.690688,48.752914,-121.026822,1659225600
1,0.690688,48.740224,-121.039512,1659225600
2,0.690688,48.740224,-121.026822,1659225600
3,0.690688,48.740224,-121.014133,1659225600
4,0.690688,48.740224,-120.976064,1659225600
...,...,...,...,...
41689,0.028940,45.631297,-122.676457,1659571200
41690,0.028940,45.631297,-122.663768,1659571200
41691,0.028940,45.618608,-122.689147,1659571200
41692,0.028940,45.618608,-122.676457,1659571200


In [195]:
all_predictions['date'] = all_predictions['dateUnixTimestamp']
all_predictions.drop(columns='dateUnixTimestamp', inplace=True)
all_predictions

,dayConfidence,latitude,longitude,date
0,0.690688,48.752914,-121.026822,1659225600
1,0.690688,48.740224,-121.039512,1659225600
2,0.690688,48.740224,-121.026822,1659225600
3,0.690688,48.740224,-121.014133,1659225600
4,0.690688,48.740224,-120.976064,1659225600
...,...,...,...,...
41689,0.028940,45.631297,-122.676457,1659571200
41690,0.028940,45.631297,-122.663768,1659571200
41691,0.028940,45.618608,-122.689147,1659571200
41692,0.028940,45.618608,-122.676457,1659571200


In [196]:
all_predictions.set_index(['date','latitude', 'longitude'], inplace=True)
all_predictions

dayConfidence
date       latitude  longitude                 
1659225600 48.752914 -121.026822       0.690688
           48.740224 -121.039512       0.690688
                     -121.026822       0.690688
                     -121.014133       0.690688
                     -120.976064       0.690688
...                                         ...
1659571200 45.631297 -122.676457       0.028940
                     -122.663768       0.028940
           45.618608 -122.689147       0.028940
                     -122.676457       0.028940
                     -122.663768       0.028940

[41694 rows x 1 columns]

# Historical Data

In [227]:
historical_data = pd.read_csv('../notebooks/processed_wa_fire_2022_July.csv' )
historical_data

,Unnamed: 0,InitialLatitude,InitialLongitude,FireDiscoveryDateTime,date,nearest_lat,nearest_lon,nearest_index
0,0,48.569070,-118.10940,2022/07/05 01:47:59+00,2022/07/05,48.575261,-118.108238,15124
1,1,47.803720,-117.10540,2022/07/05 02:19:56+00,2022/07/05,47.801202,-117.105767,46069
2,2,46.182840,-119.87170,2022/07/05 02:41:00+00,2022/07/05,46.176946,-119.872078,116796
3,3,48.193800,-120.50170,2022/07/05 15:09:54+00,2022/07/05,48.194576,-120.506553,28261
4,4,48.281330,-118.33200,2022/07/05 06:41:00+00,2022/07/05,48.283403,-118.336649,25132
...,...,...,...,...,...,...,...,...
198,198,46.958210,-122.19040,2022/07/30 16:59:06+00,2022/07/30,46.963695,-122.194256,82600
199,199,48.997167,-122.71610,2022/07/13 17:45:47+00,2022/07/13,48.994014,-122.714526,157
200,200,48.996960,-122.71610,2022/07/13 22:46:11+00,2022/07/13,48.994014,-122.714526,157
201,201,48.763420,-122.62890,2022/07/15 15:19:00+00,2022/07/15,48.765603,-122.625699,8295


In [228]:
historical_data["dateUnixTimestamp"] = historical_data.apply( lambda r: int(time.mktime(datetime.datetime.strptime(r['date'], "%Y/%m/%d").timetuple())) , axis=1 )
# historical_data = historical_data['InitialLatitude', 'InitialLongitude', 'date', 'dateUnixTimestamp']
historical_data['dayConfidence'] = '-1'
historical_data.sort_values(by='dateUnixTimestamp', inplace=True)
historical_data = historical_data[['InitialLatitude', 'InitialLongitude','dayConfidence', 'dateUnixTimestamp', 'date']]
historical_data.columns = pd.Series(['latitude', 'longitude', 'dayConfidence', 'dateUnixTimestamp', 'date'])
historical_data

,latitude,longitude,dayConfidence,dateUnixTimestamp,date
0,48.569070,-118.10940,-1,1656979200,2022/07/05
1,47.803720,-117.10540,-1,1656979200,2022/07/05
2,46.182840,-119.87170,-1,1656979200,2022/07/05
3,48.193800,-120.50170,-1,1656979200,2022/07/05
4,48.281330,-118.33200,-1,1656979200,2022/07/05
...,...,...,...,...,...
184,48.711200,-118.13158,-1,1659139200,2022/07/30
183,47.515530,-121.06380,-1,1659139200,2022/07/30
180,47.180637,-120.59284,-1,1659139200,2022/07/30
198,46.958210,-122.19040,-1,1659139200,2022/07/30


In [229]:
# drop dates above 25th

all_historical = historical_data[historical_data['date'] <= '2022/07/25']
all_historical = all_historical.drop(columns='date')
all_historical = all_historical.rename({'dateUnixTimestamp':'date'}, axis=1)
all_historical = all_historical.set_index(['date', 'latitude', 'longitude'])
all_historical

dayConfidence
date       latitude longitude              
1656979200 48.56907 -118.1094            -1
           47.80372 -117.1054            -1
           46.18284 -119.8717            -1
           48.19380 -120.5017            -1
           48.28133 -118.3320            -1
...                                     ...
1658707200 48.32681 -117.8802            -1
           47.65054 -117.4458            -1
           47.79831 -121.2913            -1
           48.03865 -118.8887            -1
           47.18523 -123.1434            -1

[161 rows x 1 columns]

# Weather data

In [230]:
weather_july = pd.read_csv('../www/flaskapp/static/20220731_predictweather.csv')
weather_july

,id,latitude,longitude,temp0,wind_speed0,precipitation0,soil_moisture0,temp1,wind_speed1,precipitation1,...,precipitation28,soil_moisture28,temp29,wind_speed29,precipitation29,soil_moisture29,temp30,wind_speed30,precipitation30,soil_moisture30
0,0,49.006704,-122.727215,66.0,1.342596,0.0,21.38,68.0,1.248693,0.0,...,0.0,20.07,82.0,2.342153,0.0,19.86,81.0,2.209168,0.0,19.67
1,1,49.006704,-122.714526,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
2,2,49.006704,-122.676457,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
3,3,49.006704,-122.625699,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
4,4,49.006704,-122.613010,62.0,1.940375,0.0,19.89,66.0,2.031342,0.0,...,0.0,17.03,76.0,1.605325,0.0,16.86,74.0,1.728974,0.0,16.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129799,129799,45.555160,-122.245014,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07
129800,129800,45.542471,-122.359220,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07
129801,129801,45.542471,-122.333841,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07
129802,129802,45.542471,-122.321151,69.0,3.152320,0.0,21.58,69.0,2.931181,0.0,...,0.0,19.32,86.0,2.297099,0.0,19.20,84.0,2.863446,0.0,19.07


In [204]:
cur_date = datetime.date(2022, 7, 1)
delta = datetime.timedelta(days=1)

weather_july_list=  []

for i in range(30):
    cur_day = weather_july[[f'temp{i}', f'wind_speed{i}', f'precipitation{i}', f'soil_moisture{i}', 'latitude', 'longitude']]
    cur_date_col = pd.DataFrame([cur_date]*len(cur_day), columns=['date'])
    cur_day = cur_day.join(cur_date_col)
    cur_date += delta
    cur_day.columns = pd.Series(['temp', 'wind_speed', 'precipitation', 'soil_moisture', 'latitude', 'longitude', 'date'])
    weather_july_list.append(cur_day)
    


In [205]:
all_weather = pd.concat(weather_july_list)
all_weather['date'] = all_weather.apply(lambda r: int(time.mktime(r['date'].timetuple())), axis=1)

In [206]:
all_weather.set_index(['date', 'latitude', 'longitude'], inplace=True)
all_weather

temp  wind_speed  precipitation  \
date       latitude  longitude                                      
1656633600 49.006704 -122.727215  66.0    1.342596            0.0   
                     -122.714526  62.0    1.940375            0.0   
                     -122.676457  62.0    1.940375            0.0   
                     -122.625699  62.0    1.940375            0.0   
                     -122.613010  62.0    1.940375            0.0   
...                                ...         ...            ...   
1659139200 45.555160 -122.245014  86.0    2.297099            0.0   
           45.542471 -122.359220  86.0    2.297099            0.0   
                     -122.333841  86.0    2.297099            0.0   
                     -122.321151  86.0    2.297099            0.0   
                     -122.308462  86.0    2.297099            0.0   

                                  soil_moisture  
date       latitude  longitude                   
1656633600 49.006704 -122.727215          21.38  
                     -122.714526          19.89  
                     -122.676457          19.89  
                     -122.625699          19.89  
                     -122.613010          19.89  
...                                         ...  
1659139200 45.555160 -122.245014          19.20  
           45.542471 -122.359220          19.20  
                     -122.333841          19.20  
                     -122.321151          19.20  
                     -122.308462          19.20  

[3894120 rows x 4 columns]

# Data assimilation

In [220]:
all_predictions

dayConfidence
date       latitude  longitude                 
1659225600 48.752914 -121.026822       0.690688
           48.740224 -121.039512       0.690688
                     -121.026822       0.690688
                     -121.014133       0.690688
                     -120.976064       0.690688
...                                         ...
1659571200 45.631297 -122.676457       0.028940
                     -122.663768       0.028940
           45.618608 -122.689147       0.028940
                     -122.676457       0.028940
                     -122.663768       0.028940

[41694 rows x 1 columns]

In [221]:
all_historical

dayConfidence
date       latitude longitude              
1656979200 48.56907 -118.1094            -1
           47.80372 -117.1054            -1
           46.18284 -119.8717            -1
           48.19380 -120.5017            -1
           48.28133 -118.3320            -1
...                                     ...
1658707200 48.32681 -117.8802            -1
           47.65054 -117.4458            -1
           47.79831 -121.2913            -1
           48.03865 -118.8887            -1
           47.18523 -123.1434            -1

[161 rows x 1 columns]

In [222]:
all_weather

temp  wind_speed  precipitation  \
date       latitude  longitude                                      
1656633600 49.006704 -122.727215  66.0    1.342596            0.0   
                     -122.714526  62.0    1.940375            0.0   
                     -122.676457  62.0    1.940375            0.0   
                     -122.625699  62.0    1.940375            0.0   
                     -122.613010  62.0    1.940375            0.0   
...                                ...         ...            ...   
1659139200 45.555160 -122.245014  86.0    2.297099            0.0   
           45.542471 -122.359220  86.0    2.297099            0.0   
                     -122.333841  86.0    2.297099            0.0   
                     -122.321151  86.0    2.297099            0.0   
                     -122.308462  86.0    2.297099            0.0   

                                  soil_moisture  
date       latitude  longitude                   
1656633600 49.006704 -122.727215          21.38  
                     -122.714526          19.89  
                     -122.676457          19.89  
                     -122.625699          19.89  
                     -122.613010          19.89  
...                                         ...  
1659139200 45.555160 -122.245014          19.20  
           45.542471 -122.359220          19.20  
                     -122.333841          19.20  
                     -122.321151          19.20  
                     -122.308462          19.20  

[3894120 rows x 4 columns]

In [223]:
all_historical =  all_historical.join(all_weather)
all_historical

dayConfidence  temp  wind_speed  precipitation  \
date       latitude longitude                                                  
1656979200 48.56907 -118.1094            -1   NaN         NaN            NaN   
           47.80372 -117.1054            -1   NaN         NaN            NaN   
           46.18284 -119.8717            -1   NaN         NaN            NaN   
           48.19380 -120.5017            -1   NaN         NaN            NaN   
           48.28133 -118.3320            -1   NaN         NaN            NaN   
...                                     ...   ...         ...            ...   
1658707200 48.32681 -117.8802            -1   NaN         NaN            NaN   
           47.65054 -117.4458            -1   NaN         NaN            NaN   
           47.79831 -121.2913            -1   NaN         NaN            NaN   
           48.03865 -118.8887            -1   NaN         NaN            NaN   
           47.18523 -123.1434            -1   NaN         NaN            NaN   

                               soil_moisture  
date       latitude longitude                 
1656979200 48.56907 -118.1094            NaN  
           47.80372 -117.1054            NaN  
           46.18284 -119.8717            NaN  
           48.19380 -120.5017            NaN  
           48.28133 -118.3320            NaN  
...                                      ...  
1658707200 48.32681 -117.8802            NaN  
           47.65054 -117.4458            NaN  
           47.79831 -121.2913            NaN  
           48.03865 -118.8887            NaN  
           47.18523 -123.1434            NaN  

[161 rows x 5 columns]

In [52]:
features = all_predictions.apply(
    lambda row: Feature(properties={"id": row["SNo"], "mag": row["dayConfidence"], "time": row["dateUnixTimestamp"]}, geometry=Point((float(row["longitude"]), float(row["latitude"])))), axis=1
).tolist()

In [53]:
feature_collection = FeatureCollection(features=features, properties={"name": "single instance predictions"})

In [56]:
with open('/home/tejabalu/www/flaskapp/static/single_instance.geojson', 'w', encoding='utf-8') as f:
    f.write(json.dumps(feature_collection, ensure_ascii=False))


In [111]:
type(all_predictions["date"][0])

pandas.core.series.Series

In [113]:
import json
df = pd.DataFrame(
    [["a", "b"], ["c", "d"]],
    index=["row 1", "row 2"],
    columns=["col 1", "col 2"],
)
df

,col 1,col 2
row 1,a,b
row 2,c,d


In [116]:
result = df.to_json(orient="split")
parsed = json.loads(result)
json.dumps(parsed)  

'{"columns": ["col 1", "col 2"], "index": ["row 1", "row 2"], "data": [["a", "b"], ["c", "d"]]}'

In [117]:
result = df.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed)  

'{"row 1": {"col 1": "a", "col 2": "b"}, "row 2": {"col 1": "c", "col 2": "d"}}'

In [118]:
result = df.to_json(orient="columns")
parsed = json.loads(result)
json.dumps(parsed)  

'{"col 1": {"row 1": "a", "row 2": "c"}, "col 2": {"row 1": "b", "row 2": "d"}}'

In [130]:
df = pd.DataFrame([{'name': 'test', 'lat': 24.13604, 'lng': 120.6778}, {'name': 'test', 'lat': 24.13604, 'lng': 120.6778}, {'name': 'test', 'lat': 24.13604, 'lng': 120.6778}])
df

,name,lat,lng
0,test,24.13604,120.6778
1,test,24.13604,120.6778
2,test,24.13604,120.6778


In [131]:
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [147]:
# columns used for constructing geojson object
import time
print(type(cur_date))
curtimeunix = int(time.mktime(cur_date.timetuple())*1000)
print(curtimeunix)

features = df.apply( 
    lambda row: Feature(properties={"id": 1, "mag": 0.1, "time": curtimeunix}, geometry=Point((float(row['lng']), float(row['lat'])))), axis=1).tolist()

# all the other columns used as properties

# whole geojson object
feature_collection = FeatureCollection(features=features, properties={"name": "single instance predictions"})

<class 'datetime.date'>
1658707200000


In [148]:
json.dumps(feature_collection)

'{"type": "FeatureCollection", "properties": {"name": "single instance predictions"}, "features": [{"type": "Feature", "geometry": {"type": "Point", "coordinates": [120.6778, 24.13604]}, "properties": {"id": 1, "mag": 0.1, "time": 1658707200000}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [120.6778, 24.13604]}, "properties": {"id": 1, "mag": 0.1, "time": 1658707200000}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [120.6778, 24.13604]}, "properties": {"id": 1, "mag": 0.1, "time": 1658707200000}}]}'